<a href="https://colab.research.google.com/github/DaviGomesti/MBA_Machine-Learning/blob/main/Machine_Learning_preparando_dados_regress%C3%A3o_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#INSTALAR O SPARK NO COLAB
!pip install pyspark py4j

#IMPORTAR SPARK NO COLAB
from pyspark.sql import SparkSession

#CRIAR SESSÃO SPARK
spark = SparkSession.builder.appName("ExemploPySpark").getOrCreate()

#OBTER CONTEXTO SPARK PARA A SESSÃO
sc = spark.sparkContext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=d37087803d7077dc9cfe2783eb8cef2ab9798ca8647ec2e4cc3faba65886c178
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
#IMPORTAR BIBLIOTECAS
from pyspark.ml.regression import LinearRegression, RandomForestRegressor

In [ ]:
#IMPORTAR BIBLIOTECAS PARA AVALIAR PERFORMANCE DO MODELO
from pyspark.ml.evaluation import  RegressionEvaluator

In [ ]:
#IMPORTAR BIBLIOTECAS HOTENDCODING
from pyspark.ml.feature import VectorAssembler

In [ ]:
#INGESTÃO DE DADOS
from pyspark.sql.types import *

In [ ]:
#IMPORT DADOS
carros_temp = spark.read.csv('/content/Machine Learning/Carros.csv', inferSchema=True, header= True, sep=';')
carros_temp.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [ ]:
#VAMOS PREVER A POTÊNCIA DO CARRO

In [ ]:
#SEPARAR AS COLUNAS DESEJADAS PARA O MODELO
carros = carros_temp.select('Consumo', 'Cilindros', 'Cilindradas', 'HP')
carros.show(5)

+-------+---------+-----------+---+
|Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
|     21|        6|        160|110|
|     21|        6|        160|110|
|    228|        4|        108| 93|
|    214|        6|        258|110|
|    187|        8|        360|175|
+-------+---------+-----------+---+
only showing top 5 rows



In [ ]:
#CRIAR O VETOR DE CARACTERISTICAS (VARIÁVEIS INDEPENDENTES)

In [ ]:
#DEFINIR AS VARIÁVEIS INDEPENDENTES NO PARAMETRO
veccaracteristicas = VectorAssembler(inputCols=[('Consumo'), ('Cilindros'), ('Cilindradas')], outputCol='caracteristicas')


In [ ]:
#AGORA VAMOS APLICAR O OBJETO CRIADO A VARIAVEL CARROS
carros = veccaracteristicas.transform(carros)
carros.show(5)

+-------+---------+-----------+---+-----------------+
|Consumo|Cilindros|Cilindradas| HP|  caracteristicas|
+-------+---------+-----------+---+-----------------+
|     21|        6|        160|110| [21.0,6.0,160.0]|
|     21|        6|        160|110| [21.0,6.0,160.0]|
|    228|        4|        108| 93|[228.0,4.0,108.0]|
|    214|        6|        258|110|[214.0,6.0,258.0]|
|    187|        8|        360|175|[187.0,8.0,360.0]|
+-------+---------+-----------+---+-----------------+
only showing top 5 rows



In [ ]:
#DIVIDIR DADOS EM TREINO E TESTE
carrosTreino, carrosTeste = carros.randomSplit([0.7, 0.3])
print(carrosTreino.count())

23


In [ ]:
print(carrosTeste.count())

9
